In [1]:
import pandas as pd

# import matplotlib.pyplot as plt

import sklearn
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn_pandas import DataFrameMapper

from keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.metrics import mean_squared_error




Using TensorFlow backend.


# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("Resources/03_new_df.csv")
df.describe()

,Opening,Theaters,Budget,Runtime,Genre_Action,Genre_Adventure,Genre_Animation,Genre_Comedy,Genre_Crime,Genre_Documentary,...,Writer_Yvette Kaplan,Writer_Zach Helm,Writer_Zach Shields,Writer_Zack Snyder,Writer_Zack Stentz,Writer_Zak Penn,Writer_Zane Smith,Writer_Zeke Richardson,Writer_Álvaro Rodríguez,Writer_Éric Toledano
count,1.907000e+03,1907.000000,1.907000e+03,1907.000000,1907.000000,1907.000000,1907.000000,1907.000000,1907.000000,1907.000000,...,1907.000000,1907.000000,1907.000000,1907.000000,1907.000000,1907.000000,1907.000000,1907.000000,1907.000000,1907.000000
mean,2.792583e+07,2913.979549,6.206534e+07,110.750393,0.359203,0.280545,0.093865,0.409544,0.149449,0.004719,...,0.000524,0.000524,0.000524,0.002622,0.001573,0.004719,0.000524,0.000524,0.000524,0.000524
std,2.729695e+07,702.092481,5.495635e+07,18.674676,0.479893,0.449384,0.291717,0.491879,0.356624,0.068554,...,0.022899,0.022899,0.022899,0.051151,0.039642,0.068554,0.022899,0.022899,0.022899,0.022899
min,1.000198e+07,629.000000,0.000000e+00,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.339860e+07,2503.000000,2.400000e+07,97.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.896668e+07,2950.000000,4.600000e+07,108.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3.053719e+07,3389.500000,8.000000e+07,122.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3.571150e+08,4662.000000,5.000000e+08,201.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [3]:
df.head()

,Studio,Opening,Theaters,Date,Budget,Runtime,Rating,Genre_Action,Genre_Adventure,Genre_Animation,...,Writer_Yvette Kaplan,Writer_Zach Helm,Writer_Zach Shields,Writer_Zack Snyder,Writer_Zack Stentz,Writer_Zak Penn,Writer_Zane Smith,Writer_Zeke Richardson,Writer_Álvaro Rodríguez,Writer_Éric Toledano
0,BV,357115007,4662,2019-04-26,356000000,181.0,PG-13,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,BV,257698183,4474,2018-04-27,300000000,149.0,PG-13,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,BV,247966675,4134,2015-12-18,245000000,136.0,PG-13,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,BV,220009584,4232,2017-12-15,200000000,152.0,PG-13,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,Uni.,208806270,4274,2015-06-12,150000000,124.0,PG-13,1,1,0,...,0,0,0,0,0,0,0,0,0,0


# Change Date

In [4]:
date_column = df["Date"].str.split("-", n = 2, expand = True) 
year_col= date_column[0] 
month_col= date_column[1] 
day_col= date_column[2] 

In [5]:
# one_hot_year = OneHotEncoder(dtype=np.int, categories="auto", sparse=False).fit_transform(year_col.values.reshape(-1,1))
# one_hot_year = pd.Series(list(one_hot_year))
# # Create DataFrame 
# new_year_df = pd.DataFrame(one_hot_year, columns=["Year"]) 

In [6]:
# one_hot_month = OneHotEncoder(dtype=np.int, categories="auto", sparse=False).fit_transform(month_col.values.reshape(-1,1))
# one_hot_month = pd.Series(list(one_hot_month))
# new_month_df = pd.DataFrame(one_hot_month, columns=["Month"]) 

In [7]:
# one_hot_day = OneHotEncoder(dtype=np.int, categories="auto", sparse=False).fit_transform(day_col.values.reshape(-1,1))
# one_hot_day = pd.Series(list(one_hot_day))
# new_day_df = pd.DataFrame(one_hot_day, columns=["Day"]) 

In [8]:
year_col.describe()

count     1907
unique      40
top       2012
freq        89
Name: 0, dtype: object

In [9]:
label_encoder = LabelEncoder()
label_encoder.fit(year_col)
encoded_year = label_encoder.transform(year_col)

In [10]:
encoded_year

array([39, 38, 35, ..., 26, 34, 32])

In [11]:
one_hot_year = to_categorical(encoded_year)
one_hot_year

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [12]:
one_hot_year.shape

(1907, 40)

In [13]:
one_hot_month = to_categorical(month_col)
one_hot_month

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [14]:
one_hot_month.shape

(1907, 13)

In [15]:
one_hot_day = to_categorical(day_col)
one_hot_day

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [16]:
one_hot_day.shape

(1907, 32)

In [17]:
year_df = pd.DataFrame(one_hot_year).add_prefix('Year_')

In [18]:
month_df = pd.DataFrame(one_hot_month).add_prefix('Month_')

In [19]:
day_df = pd.DataFrame(one_hot_day).add_prefix('Day_')

In [20]:
# date_df = pd.concat([year_df, month_df, day_df], axis=1)
# trying it without YEAR
date_df = pd.concat([month_df, day_df], axis=1)
date_df.shape

(1907, 45)

In [21]:
df = df.drop(columns=["Date"])
df = pd.get_dummies(df)
df.head()

,Opening,Theaters,Budget,Runtime,Genre_Action,Genre_Adventure,Genre_Animation,Genre_Comedy,Genre_Crime,Genre_Documentary,...,Studio_Uni.,Studio_W/Dim.,Studio_WB,Studio_WB (NL),Studio_Wein.,Rating_G,Rating_NR,Rating_PG,Rating_PG-13,Rating_R
0,357115007,4662,356000000,181.0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,257698183,4474,300000000,149.0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,247966675,4134,245000000,136.0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,220009584,4232,200000000,152.0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,208806270,4274,150000000,124.0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0


In [22]:
df = pd.concat([df, date_df], axis=1)
df.head()

,Opening,Theaters,Budget,Runtime,Genre_Action,Genre_Adventure,Genre_Animation,Genre_Comedy,Genre_Crime,Genre_Documentary,...,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30,Day_31
0,357115007,4662,356000000,181.0,1,1,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,257698183,4474,300000000,149.0,1,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,247966675,4134,245000000,136.0,1,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,220009584,4232,200000000,152.0,1,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,208806270,4274,150000000,124.0,1,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Create a Train Test Split
Use `Opening` for the y values

In [23]:
target = df["Opening"]
data = df.drop("Opening", axis=1)
feature_names = data.columns
data.head()

,Theaters,Budget,Runtime,Genre_Action,Genre_Adventure,Genre_Animation,Genre_Comedy,Genre_Crime,Genre_Documentary,Genre_Drama,...,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30,Day_31
0,4662,356000000,181.0,1,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,4474,300000000,149.0,1,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,4134,245000000,136.0,1,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4232,200000000,152.0,1,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4274,150000000,124.0,1,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [25]:
X_train.shape

(1430, 16314)

In [26]:
y_train.shape

(1430,)

# Pre-processing

Scale the data using the MinMaxScaler

In [27]:
X_minmax = MinMaxScaler().fit(X_train)
# y_train.values.reshape(-1, 1)
y_minmax = MinMaxScaler().fit(y_train.values.reshape(-1, 1))

In [28]:
X_train_scaled = X_minmax.transform(X_train)
X_test_scaled = X_minmax.transform(X_test)
y_train_scaled = y_minmax.transform(y_train.values.reshape(-1, 1))
y_test_scaled = y_minmax.transform(y_test.values.reshape(-1, 1))

In [29]:
# X_mapper = DataFrameMapper([(['Theaters', 'Budget', 'Runtime'], MinMaxScaler())], 
#                           default=None).fit(X_train)

In [30]:
# X_train_scaled = X_mapper.transform(X_train)
# X_test_scaled = X_mapper.transform(X_test)

In [31]:
X_train_scaled.shape

(1430, 16314)

In [32]:
X_train_scaled

array([[0.56586366, 0.06      , 0.45142857, ..., 0.        , 0.        ,
        0.        ],
       [0.67657611, 0.32      , 0.55428571, ..., 0.        , 0.        ,
        0.        ],
       [0.61455664, 0.08      , 0.51428571, ..., 0.        , 1.        ,
        0.        ],
       ...,
       [0.21655561, 0.05      , 0.56571429, ..., 0.        , 0.        ,
        0.        ],
       [0.63172732, 0.034     , 0.34857143, ..., 0.        , 0.        ,
        0.        ],
       [0.65171707, 0.064     , 0.34857143, ..., 0.        , 0.        ,
        0.        ]])

In [33]:
y_train.shape

(1430,)

# Try simple linear regression

In [34]:
# # Import dependencies
# from sklearn.linear_model import LinearRegression

# # Create a linear model
# model = LinearRegression()

# # Fit (Train) our model to the data
# model.fit(X_train_scaled, y_train_scaled)

In [35]:
# predictions = model.predict(X_test_scaled)

In [36]:
# # Overall Score for the model
# model.score(X_train_scaled, y_train_scaled)

In [37]:
# model.score(X_test_scaled, y_test_scaled)

In [38]:
# MSE = mean_squared_error(y_test_scaled, predictions)
# r2 = model.score(X_test_scaled, y_test_scaled)

# print(f"MSE: {MSE}, R2: {r2}")

# LASSO model

In [39]:
# # LASSO model
# # Note: Use an alpha of .01 when creating the model for this activity
# from sklearn.linear_model import Lasso

# lasso = Lasso(alpha=.01).fit(X_train_scaled, y_train_scaled)

# predictions = lasso.predict(X_test_scaled)

# MSE = mean_squared_error(y_test_scaled, predictions)
# r2 = lasso.score(X_test_scaled, y_test_scaled)
# ### END SOLUTION

# print(f"MSE: {MSE}, R2: {r2}")

In [40]:
# lasso.score(X_train_scaled, y_train_scaled)

In [41]:
# lasso.score(X_test_scaled, y_test_scaled)

# Ridge model

In [42]:
# # Ridge model
# # Note: Use an alpha of .01 when creating the model for this activity
# from sklearn.linear_model import Ridge

# ridge = Ridge(alpha=.01).fit(X_train_scaled, y_train_scaled)

# predictions = ridge.predict(X_test_scaled)

# MSE = mean_squared_error(y_test_scaled, predictions)
# r2 = ridge.score(X_test_scaled, y_test_scaled)

# print(f"MSE: {MSE}, R2: {r2}")

In [43]:
# ridge.score(X_train_scaled, y_train_scaled)

In [44]:
# ridge.score(X_test_scaled, y_test_scaled)

# ElasticNet model

In [45]:
# # ElasticNet model
# # Note: Use an alpha of .01 when creating the model for this activity
# from sklearn.linear_model import ElasticNet

# ### BEGIN SOLUTION
# elasticnet = ElasticNet(alpha=.01).fit(X_train_scaled, y_train_scaled)
# model.score(X_train_scaled, y_train_scaled)

# model.score(X_test_scaled, y_test_scaled)
# predictions = elasticnet.predict(X_test_scaled)

# MSE = mean_squared_error(y_test_scaled, predictions)
# r2 = elasticnet.score(X_test_scaled, y_test_scaled)
# ### END SOLUTION

# print(f"MSE: {MSE}, R2: {r2}")

In [46]:
# elasticnet.score(X_train_scaled, y_train_scaled)

In [47]:
# elasticnet.score(X_test_scaled, y_test_scaled)

# Try Neural Net: Define the architecture

In [48]:
nn_model = Sequential()

In [49]:
nn_model.add(Dense(units=1500, input_dim=16314, activation='relu'))
nn_model.add(Dense(units=800, activation='relu'))
nn_model.add(Dense(units=300, activation='relu'))
nn_model.add(Dense(units=1, activation='linear'))
nn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1500)              24472500  
_________________________________________________________________
dense_1 (Dense)              (None, 800)               1200800   
_________________________________________________________________
dense_2 (Dense)              (None, 300)               240300    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 301       
Total params: 25,913,901
Trainable params: 25,913,901
Non-trainable params: 0
_________________________________________________________________


In [50]:
nn_model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])

In [51]:
nn_model.fit(
    X_train_scaled,
    y_train_scaled,
    epochs=100,
    shuffle=False,
    verbose=2
)

Train on 1430 samples
Epoch 1/100
1430/1430 - 6s - loss: 0.0069 - mse: 0.0069 - mae: 0.0520
Epoch 2/100
1430/1430 - 6s - loss: 0.0052 - mse: 0.0052 - mae: 0.0382
Epoch 3/100
1430/1430 - 7s - loss: 0.0031 - mse: 0.0031 - mae: 0.0314
Epoch 4/100
1430/1430 - 6s - loss: 0.0041 - mse: 0.0041 - mae: 0.0320
Epoch 5/100
1430/1430 - 6s - loss: 0.0067 - mse: 0.0067 - mae: 0.0351
Epoch 6/100
1430/1430 - 6s - loss: 0.0082 - mse: 0.0082 - mae: 0.0291
Epoch 7/100
1430/1430 - 5s - loss: 0.0041 - mse: 0.0041 - mae: 0.0272
Epoch 8/100
1430/1430 - 5s - loss: 0.0069 - mse: 0.0069 - mae: 0.0264
Epoch 9/100
1430/1430 - 5s - loss: 0.0016 - mse: 0.0016 - mae: 0.0190
Epoch 10/100
1430/1430 - 5s - loss: 6.4843e-04 - mse: 6.4843e-04 - mae: 0.0142
Epoch 11/100
1430/1430 - 5s - loss: 7.5800e-04 - mse: 7.5800e-04 - mae: 0.0144
Epoch 12/100
1430/1430 - 5s - loss: 0.0010 - mse: 0.0010 - mae: 0.0163
Epoch 13/100
1430/1430 - 5s - loss: 0.0015 - mse: 0.0015 - mae: 0.0181
Epoch 14/100
1430/1430 - 5s - loss: 0.0014 - mse

In [52]:
nn_model.evaluate(
    X_test_scaled, y_test_scaled, verbose=2)

477/477 - 0s - loss: 0.0020 - mse: 0.0020 - mae: 0.0298


[0.001969248973090881, 0.0019692492, 0.029752612]